In [ ]:
import astropy.units as u
import numpy as np
import warnings 
import sys 
warnings.filterwarnings('ignore')
sys.path.append('./../')

from tools_contsub_main import *
from tools_contsub_misc import *
from tools_contsub_units import *
from tools_contsub_plots import *
from tools_contsub_anchoring import * 
from tools_contsub_smoothregrid import * 
from tools_contsub_postprocess import * 

In [ ]:
galaxy = 'ngc5236'
galaxy_muse = galaxy
rootdir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/%s/' %galaxy
rootdir_bp = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/hst_filters/' 
outputdir = 'hst_contsub/'

narrowband_filter = 'f657n'
instrument_f555w  = 'uvis'
instrument_f65Xn = 'uvis'
instrument_f814w = 'uvis'

hdu_hst_f555w   = get_hdu(rootdir, 'hst/*%s_*f547m*.fits' %(instrument_f555w))
hdu_hst_f65Xn   = get_hdu(rootdir, 'hst/*%s_*%s*.fits' %(instrument_f65Xn, narrowband_filter))
hdu_hst_f814w   = get_hdu(rootdir, 'hst/*%s_*f814w*.fits' %(instrument_f814w))

# Get filter curve info
bp = get_bandpassinfo(rootdir_bp)

# Cosmic ray thresholds
cr_threshold = 0.25
cr_dilation_iterations = 1

In [ ]:
# Make paths
make_paths(rootdir, outputdir)

# Convert units 
hdu_hst_f555w = get_nanzeros(hdu_hst_f555w)
hdu_hst_f65Xn = get_nanzeros(hdu_hst_f65Xn)
hdu_hst_f814w = get_nanzeros(hdu_hst_f814w)

hdu_hst_f65Xn = remove_nan_padding(hdu_hst_f65Xn)
hdu_hst_f555w = get_regrid(hdu_hst_f555w, hdu_hst_f65Xn)
hdu_hst_f814w = get_regrid(hdu_hst_f814w, hdu_hst_f65Xn)

hdu_hst_f555w, hdu_hst_f65Xn, hdu_hst_f814w = get_covmask(hdu_hst_f555w, hdu_hst_f65Xn, hdu_hst_f814w)

hdu_hst_f555w = get_electrons_2_ergcm2sA(hdu_hst_f555w)
hdu_hst_f65Xn = get_electrons_2_ergcm2sA(hdu_hst_f65Xn)
hdu_hst_f814w = get_electrons_2_ergcm2sA(hdu_hst_f814w)

# Continuum subtract       
hdu_hst_halpha, hdu_hst_cont = get_contsub(hdu_hst_f65Xn.copy(), 
                                hdu_hst_f555w.copy(), 
                                hdu_hst_f814w.copy(), 
                                bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['pivot'], 
                                bp['%s_%s' %(instrument_f555w.upper(), 'F555W')]['pivot'], 
                                bp['%s_%s' %(instrument_f814w.upper(), 'F814W')]['pivot'])

# Convert units
photbw = bp['%s_%s' %(instrument_f65Xn.upper(), narrowband_filter.upper())]['rectwidth']
hdu_hst_halpha = get_ergcm2sA_2_ergcm2s(hdu_hst_halpha, photbw)
hdu_hst_cont = get_ergcm2sA_2_ergcm2s(hdu_hst_cont, photbw)

# Make maps plots
make_plots_map(hdu_hst_halpha, galaxy, 'hst_%s_contsub' %narrowband_filter, rootdir)

# Save files
write_hdu(hdu_hst_halpha, rootdir, '%s_hst_%s_contsub.fits' %(galaxy, narrowband_filter))
write_hdu(hdu_hst_cont, rootdir, '%s_hst_%s_cont.fits' %(galaxy, narrowband_filter))

In [ ]:
def get_anchoring(hdu1, hdu2, hdu3, filter='', make_plots=True):

    ### 
    hdu1 = hdu1.copy()
    hdu2 = hdu2.copy()
    hdu3 = hdu3.copy()
    data1 = hdu1.data.copy()
    data2 = hdu2.data.copy()
    data3 = hdu3.data.copy()

    # Mask zeros 
    mask_zero1 = data1==0
    mask_zero2 = data2==0
    data1[(mask_zero1&mask_zero2)] = np.nan
    data2[(mask_zero1&mask_zero2)] = np.nan

    valid_indices = np.isfinite(data1) & np.isfinite(data2)
    data1 = data1[valid_indices]
    data2 = data2[valid_indices]

    # Mask to only lowest value points 
    x_per = np.percentile(data1, [0.1, 99.9])
    y_per = np.percentile(data2, [0.1, 99.9])

    x_mask = (data1>x_per[0])&(data1<x_per[1])
    y_mask = (data2>y_per[0])&(data2<y_per[1])

    data1 = data1[x_mask&y_mask]
    data2 = data2[x_mask&y_mask]

    # Get bins with equal number of points in each bin 
    min_val, max_val = np.percentile(data1, [10, 90]) 
    # bin_values = get_bins(data1, data2, 20, equal_spaced=True, min_val=min_val, max_val=max_val)
    bin_values = get_bins(data1, data2, 20, equal_spaced=False, min_val=min_val, max_val=max_val)

    # Fit binned data
    model_poly = models.Polynomial1D(degree=1)
    fitter_poly = fitting.LinearLSQFitter() 
    best_fit_poly_bins = fitter_poly(model_poly, bin_values[0], bin_values[1])
    intercept_bins, slope_bins = best_fit_poly_bins.parameters

    x_fit = np.linspace(-1e3, np.nanmax(data2), 10000)
    y_fit_bins = slope_bins * x_fit + intercept_bins
    ###

    # Extract the WCS information from the input and template headers
    wcs1 = wcs.WCS(hdu1.header)
    wcs3 = wcs.WCS(hdu3.header)
    pixscale1 = wcs.utils.proj_plane_pixel_area(wcs1.celestial)
    pixscale3 = wcs.utils.proj_plane_pixel_area(wcs3.celestial)

    pixscale_ratio = (pixscale3 / pixscale1)
    offset1 = (0 - (intercept_bins*pixscale_ratio)) / slope_bins
    offset2 = (0 - (intercept_bins)) / slope_bins

    hdu3.data = (hdu3.data - (intercept_bins*pixscale_ratio)) / slope_bins 
    hdu2.data = (hdu2.data - (intercept_bins)) / slope_bins 

    fit = [filter, slope_bins, intercept_bins, intercept_bins*pixscale_ratio, offset1, offset2]
    table_fit = Table(np.array(fit), names=['filter', 'slope_bins', 'intercept_lowres', 'intercept_highres', 'offset_lowres', 'offset_highres'])
    #### 

    if make_plots: 

        fig = plt.figure(figsize=(10, 5))
        ax1 = fig.add_subplot(1, 2, 1)
        ax2 = fig.add_subplot(1, 2, 2)

        for ax in [ax1,ax2]:

            #data
            ax.scatter(data1, data2, c='k', alpha=0.01, s=1, rasterized=True)

            #bins 
            ax.scatter(bin_values[0], bin_values[1], fc='none', ec='C0', alpha=1, s=30, zorder=5)
            ax.plot(bin_values[0], bin_values[1], c='C0', alpha=1, zorder=5)

            # fits 
            offset = (0 - (intercept_bins)) / slope_bins
            ax.plot(x_fit, y_fit_bins, color='C0', linewidth=2, linestyle=':', label=f'y = {slope_bins:.4f}x + {intercept_bins:.4g}')
            ax.plot(x_fit, x_fit, 'k', linewidth=2, linestyle=':', label=f'y = x')
            ax.plot([offset, offset], [-100,0], color='C0', linewidth=2, linestyle=':', label=f'Offset = {offset:.4g}')

            for f in [0.1,0.5,2,10]:
                ax.plot(x_fit, x_fit*f, 'k', linewidth=2, linestyle='--', alpha=0.1)

            ax.set_xlabel('Flux (MUSE Ha) [erg/s/cm-2/pix]')
            ax.set_ylabel('Flux (MUSE contsub) [erg/s/cm-2/pix]')
            ax.legend(title=filter, loc='upper left')
            ax.grid(True, ls=':', color='k', alpha=0.2)

        ax1.set_xlim(np.nanpercentile(data1, [0,99]))
        ax1.set_ylim(np.nanpercentile(data2, [0,99]))

        ax2.set_xlim(np.nanpercentile(data1[data1>0], [0.01,99.99]))
        ax2.set_ylim(np.nanpercentile(data2[data2>0], [0.01,99.99]))

        ax2.set_xscale('log')
        ax2.set_yscale('log')

        plt.tight_layout()

    return(hdu3, hdu2, table_fit)

In [ ]:
def get_regrid(hdu_input, hdu_template, output_filename=None, conserve_flux=True, order='bilinear'):

    print("[INFO] Reprojecting the input image to match the template WCS...")

    # Extract the WCS information from the input and template headers
    header_input = hdu_input.header
    header_template = hdu_template.header
    wcs_input = wcs.WCS(header_input)
    wcs_template = wcs.WCS(header_template)

    # Calculate the pixel scale for input and template images
    pixscale_input = wcs.utils.proj_plane_pixel_area(wcs_input.celestial)
    pixscale_template = wcs.utils.proj_plane_pixel_area(wcs_template.celestial)

    # Reproject the input image to match the template WCS
    print("[INFO] Performing image reprojection...")
    data_output = reproject_interp(hdu_input, header_template, order=order)[0]
    print("[INFO] Image reprojection complete.")

    keys = ['NAXIS', 'NAXIS1', 'NAXIS2', 'CRPIX1', 'CRPIX2', 'CRVAL1', 'CRVAL2', 'CD1_1', 'CD2_2']
    for key in keys: 
        header_input[key] = header_template[key]
    hdu_output = fits.PrimaryHDU(data_output, header_input)

    if conserve_flux:
        # Scale the output data to conserve flux 
        print(f"[INFO] Scaling the output data to conserve flux with factor {(pixscale_template / pixscale_input):.2f}")
        hdu_output.data = hdu_output.data * (pixscale_template / pixscale_input)
        hdu_output.data = np.array(hdu_output.data, dtype=np.float32)
        print("[INFO] Flux scaling complete.")

    print("[INFO] Reprojection process completed.")
    return(hdu_output)

In [ ]:
hdu_muse =  fits.open(rootdir+'muse/M83_MAPS.fits')
hdu_muse_f65Xn = hdu_muse['HA6562_FLUX']

# Get resolution 
hst_res  = 0.07 * u.arcsec
muse_res = 0.86 * u.arcsec

# Smooth and regrid to MUSE
hdu_hst_halpha_sm = get_smooth(hdu_hst_halpha, hst_res, muse_res)
hdu_hst_halpha_smre = get_regrid(hdu_hst_halpha_sm, hdu_muse_f65Xn)

hdu_hst_halpha_scaled, hdu_muse_halpha_scaled, _ = get_anchoring(hdu_muse_f65Xn, hdu_hst_halpha_smre, hdu_hst_halpha)

# Save files
write_hdu(hdu_hst_halpha_smre, rootdir, '%s_hst_%s_contsub_smre.fits' %(galaxy, narrowband_filter))
write_hdu(hdu_hst_halpha_scaled, rootdir, '%s_hst_%s_contsub_scaled.fits' %(galaxy, narrowband_filter), appdir=outputdir)

In [ ]:
# # Coverage mask
# hdu_mask = get_mask(hdu_hst_halpha)

# # Interpolate negatives
# hdu_hst_halpha_i = get_interp_negs(hdu_hst_halpha, hdu_mask)

# # Remove cosmic rays
# hdu_hst_halpha_ic = get_cosmicrays(hdu_hst_halpha_i, hdu_mask, threshold=cr_threshold, dilation_iterations=cr_dilation_iterations)

# # Save files
# write_hdu(hdu_hst_halpha_i, rootdir, '%s_hst_%s_contsub_i.fits' %(galaxy, narrowband_filter))
# write_hdu(hdu_hst_halpha_ic, rootdir, '%s_hst_%s_contsub_ic.fits' %(galaxy, narrowband_filter))